#### Friday, December 8, 2023

[PAL : Program-aided Language Models with LangChain code](https://www.youtube.com/watch?v=dy7-LvDu-3s&list=PL8motc6AQftk1Bs42EW45kwYbyJ4jOdiZ&index=7)

In [ ]:
# !pip -q install openai==0.27.0 langchain huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.6/360.6 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 5.0 MB/s eta 0:00:00


## PAL

https://langchain.readthedocs.io/en/latest/modules/chains/examples/pal.html?highlight=PAL

Implements Program-Aided Language Models, as in https://arxiv.org/pdf/2211.10435.pdf.

https://reasonwithpal.com/

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""


Yeah, we are not going to use OpenAI ... we are gonna use [TheBloke/Python-Code-33B-GPTQ](https://huggingface.co/TheBloke/Python-Code-33B-GPTQ)

In [1]:
!ls /home/rob/Data2/huggingface/transformers

models--Intel--neural-chat-7b-v3-1
models--TheBloke--CodeLlama-34B-Instruct-GPTQ
models--TheBloke--Llama-2-13b-Chat-GPTQ
models--TheBloke--Orca-2-13B-AWQ
models--TheBloke--Orca-2-13B-GPTQ
models--TheBloke--Python-Code-33B-GPTQ
models--facebook--blenderbot-1B-distill
models--google--flan-t5-large
models--gpt2-medium
models--meta-llama--Llama-2-13b-chat-hf
models--meta-llama--Llama-2-13b-hf
models--model_name
tmp0m77pc0x
tmpcjh0h7gn
tmpfcrmwgx2
tmpzafytbf_
version.txt


In [2]:
# You need this to just target the 4090.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
model_name_or_path = "TheBloke/Python-Code-33B-GPTQ"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [6]:
# To use a different branch, change revision
# For example: revision="gptq-4bit-128g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

# 13.1s 
# 18424 MiB VRAM

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


In [7]:
prompt = "Tell me about AI"
prompt_template=f'''This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: {prompt}
ASSISTANT:
'''


In [8]:
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# 25.6s

<s> This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:

Artificial Intelligence (AI) is a field of computer science that studies intelligent agents. An intelligent agent is a program that can solve problems by using its own knowledge and experience. AI researchers study how to design and build such agents.

AI agents can be trained on a large dataset to identify patterns and make predictions. They can also be programmed to follow a set of rules or logic to solve problems.

Some popular AI techniques include:

1. Machine Learning: This technique involves using a large dataset to train an AI agent. The agent learns to identify patterns and make predictions based on the data.

2. Deep Learning: This technique uses a deep neural network to train an AI agent. The network consists of multiple layers that process data in a hierarchical manner.

In [9]:
# Inference can also be done using transformers' pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

In [10]:
print(pipe(prompt_template)[0]['generated_text'])

# 16.5s

This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:
Artificial Intelligence (AI) is a field of computer science that studies intelligent agents. An intelligent agent is a program that can perform tasks that require intelligence, such as solving problems, making decisions, and communicating with humans. 

AI researchers study various aspects of intelligence, such as learning, reasoning, and problem-solving. They use techniques like machine learning and deep learning to develop AI systems that can perform these tasks. 

Some popular AI applications include:

1. Speech recognition: This technology allows computers to understand human speech and respond accordingly. It is used in products like Siri, Google Assistant, and Alexa.

2. Facial recognition: This technology allows computers to identify people based on their facial features. It is u

In [11]:
!pip show langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: langchain
Version: 0.0.348
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: None
Author-email: None
License: MIT
Location: /usr/local/lib/python3.8/dist-packages
Requires: pydantic, langsmith, SQLAlchemy, async-timeout, PyYAML, requests, dataclasses-json, jsonpatch, langchain-core, tenacity, aiohttp, numpy
Required-by: langchain-experimental


In [12]:
from langchain_experimental.pal_chain.base import PALChain

In [13]:
# from langchain.chains import PALChain
#from langchain import OpenAI
from langchain.chains.llm import LLMChain

In [ ]:
# llm = OpenAI(model_name='code-davinci-002',
#              temperature=0,
#              max_tokens=512)

## PAL - Program aided Language Model

https://www.prodigygame.com/main-en/blog/math-word-problems/

<img src="https://www.dropbox.com/s/i0wiuneh61r9u28/Screenshot%202023-03-09%20at%206.39.34%20PM.png?raw=1" alt="example image" width="300">

In [16]:
pal_chain = PALChain.from_math_prompt(pipe, verbose=True)

ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [ ]:
pal_chain = PALChain.from_math_prompt(llm, verbose=True)

In [ ]:
question = "The cafeteria had 23 apples. \
If they used 20 for lunch and bought 6 more,\
how many apples do they have?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """The cafeteria had 23 apples. If they used 20 for lunch and bought 6 more,how many apples do they have?"""
    apples_initial = 23
    apples_used = 20
    apples_bought = 6
    apples_left = apples_initial - apples_used + apples_bought
    result = apples_left
    return result

> Finished chain.


'9'

In [ ]:
question

'The cafeteria had 23 apples. If they used 20 for lunch and bought 6 more,how many apples do they have?'

In [ ]:
pal_chain.prompt.template

'Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?\n\n# solution in Python:\n\n\ndef solution():\n    """Olivia has $23. She bought five bagels for $3 each. How much money does she have left?"""\n    money_initial = 23\n    bagels = 5\n    bagel_cost = 3\n    money_spent = bagels * bagel_cost\n    money_left = money_initial - money_spent\n    result = money_left\n    return result\n\n\n\n\n\nQ: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?\n\n# solution in Python:\n\n\ndef solution():\n    """Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?"""\n    golf_balls_initial = 58\n    golf_balls_lost_tuesday = 23\n    golf_balls_lost_wednesday = 2\n    golf_balls_left = golf_balls_initial - golf_balls_lost_tuesday - golf_balls_lost_wednesday\n    result =

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?

# solution in Python:


def solution():
    """Olivia has $23. She bought five bagels for $3 each. How much money does she have left?"""
    money_initial = 23
    bagels = 5
    bagel_cost = 3
    money_spent = bagels * bagel_cost
    money_left = money_initial - money_spent
    result = money_left
    return result





Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?

# solution in Python:


def solution():
    """Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?"""
    golf_balls_initial = 58
    golf_balls_lost_tuesday = 23
    golf_balls_lost_wednesday = 2
    golf_balls_left = golf_balls_initial - golf_balls_lost_tuesday - golf_balls_lost_wednesday
    result = golf_balls_left
    return result





Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?

# solution in Python:


def solution():
    """There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?"""
    computers_initial = 9
    computers_per_day = 5
    num_days = 4  # 4 days between monday and thursday
    computers_added = computers_per_day * num_days
    computers_total = computers_initial + computers_added
    result = computers_total
    return result





Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?

# solution in Python:


def solution():
    """Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?"""
    toys_initial = 5
    mom_toys = 2
    dad_toys = 2
    total_received = mom_toys + dad_toys
    total_toys = toys_initial + total_received
    result = total_toys
    return result





Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?

# solution in Python:


def solution():
    """Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?"""
    jason_lollipops_initial = 20
    jason_lollipops_after = 12
    denny_lollipops = jason_lollipops_initial - jason_lollipops_after
    result = denny_lollipops
    return result





Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?

# solution in Python:


def solution():
    """Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?"""
    leah_chocolates = 32
    sister_chocolates = 42
    total_chocolates = leah_chocolates + sister_chocolates
    chocolates_eaten = 35
    chocolates_left = total_chocolates - chocolates_eaten
    result = chocolates_left
    return result





Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?

# solution in Python:


def solution():
    """If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?"""
    cars_initial = 3
    cars_arrived = 2
    total_cars = cars_initial + cars_arrived
    result = total_cars
    return result





Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?

# solution in Python:


def solution():
    """There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?"""
    trees_initial = 15
    trees_after = 21
    trees_added = trees_after - trees_initial
    result = trees_added
    return result





Q: {question}

# solution in Python:




## GSM8k

In [ ]:
question = "John buys twice as many red ties as blue ties. The red ties cost 50% more than blue ties. \
He spent $200 on blue ties that cost $40 each. How much did he spend on ties?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """John buys twice as many red ties as blue ties. The red ties cost 50% more than blue ties. He spent $200 on blue ties that cost $40 each. How much did he spend on ties?"""
    blue_ties_cost = 40
    blue_ties_num = 200 / blue_ties_cost
    red_ties_num = blue_ties_num * 2
    red_ties_cost = blue_ties_cost * 1.5
    red_ties_total = red_ties_num * red_ties_cost
    blue_ties_total = blue_ties_num * blue_ties_cost
    total_spent = red_ties_total + blue_ties_total
    result = total_spent
    return result

> Finished chain.


'800.0'

In [ ]:
question = "Maggie spent a quarter of her money, while Riza spent one-third of her money. \
They each had $60. How much money do the two of them have left?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """Maggie spent a quarter of her money, while Riza spent one-third of her money. They each had $60. How much money do the two of them have left?"""
    maggie_money_initial = 60
    maggie_money_spent = maggie_money_initial / 4
    maggie_money_left = maggie_money_initial - maggie_money_spent
    riza_money_initial = 60
    riza_money_spent = riza_money_initial / 3
    riza_money_left = riza_money_initial - riza_money_spent
    total_money_left = maggie_money_left + riza_money_left
    result = total_money_left
    return result

> Finished chain.


'85.0'

In [ ]:
question = "In November, a toy was $8186321.4112. \
In December, the price increased by 80%. In January, the price decreased by 50%. \
What was the price of the toy after it was discounted in January?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """In November, a toy was $8186321.4112. In December, the price increased by 80%. In January, the price decreased by 50%. What was the price of the toy after it was discounted in January?"""
    price_november = 8186321.4112
    price_december = price_november * 1.8
    price_january = price_december * 0.5
    result = price_january
    return result

> Finished chain.


'7367689.27008'

## Repeat words

In [ ]:
question = "Repeat cheese seven times; every third say whiz"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """Repeat cheese seven times; every third say whiz"""
    cheese = "cheese"
    whiz = "whiz"
    cheese_count = 7
    whiz_count = 3
    cheese_result = cheese * cheese_count
    whiz_result = whiz * whiz_count
    result = cheese_result + whiz_result
    return result

> Finished chain.


'cheesecheesecheesecheesecheesecheesecheesewhizwhizwhiz'

In [ ]:
question = "Say the letters of the alphabet in capital letters, but only the odd ones"
#A C E G I K M O Q S U W Y"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """Say the letters of the alphabet in capital letters, but only the odd ones"""
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    odd_letters = alphabet[::2]
    result = odd_letters
    return result

> Finished chain.


'ACEGIKMOQSUWY'

## Word Math Problems

https://www.prodigygame.com/main-en/blog/math-word-problems/

In [ ]:
question = "If you have 80 tickets for the fair and each ride costs 5 tickets, \
how many rides can you go on?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """If you have 80 tickets for the fair and each ride costs 5 tickets, how many rides can you go on?"""
    tickets_initial = 80
    ride_cost = 5
    rides_possible = tickets_initial // ride_cost
    result = rides_possible
    return result

> Finished chain.


'16'

In [ ]:
question = "The school has $20,000 to buy new computer equipment. If each piece of equipment costs $50,\
 how many pieces can the school buy in total?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """The school has $20,000 to buy new computer equipment. If each piece of equipment costs $50, how many pieces can the school buy in total?"""
    money_available = 20000
    equipment_cost = 50
    total_equipment = money_available // equipment_cost
    result = total_equipment
    return result

> Finished chain.


'400'

In [ ]:
#interpreting remainders
question = 'An Italian restaurant receives a shipment of 86 veal cutlets. If it takes 3 cutlets to make a dish, \
how many cutlets will the restaurant have left over after making as many dishes as possible?'

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """An Italian restaurant receives a shipment of 86 veal cutlets. If it takes 3 cutlets to make a dish, how many cutlets will the restaurant have left over after making as many dishes as possible?"""
    cutlets_initial = 86
    cutlets_per_dish = 3
    num_dishes = cutlets_initial // cutlets_per_dish
    cutlets_leftover = cutlets_initial % cutlets_per_dish
    result = cutlets_leftover
    return result

> Finished chain.


'2'

In [ ]:
# mixed operations
question = "There are 235 books in a library. On Monday, 123 books are taken out. \
On Tuesday, 56 books are brought back. How many books are there now?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """There are 235 books in a library. On Monday, 123 books are taken out. On Tuesday, 56 books are brought back. How many books are there now?"""
    books_initial = 235
    books_taken_out = 123
    books_returned = 56
    books_left = books_initial - books_taken_out + books_returned
    result = books_left
    return result

> Finished chain.


'168'

In [ ]:
# percentages
question = "The school’s junior band has 10 saxophone players and 20 trumpet players. \
 The school’s senior band has 18 saxophone players and 29 trumpet players. \
Which band has the higher ratio of trumpet to saxophone players?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """The school’s junior band has 10 saxophone players and 20 trumpet players.  The school’s senior band has 18 saxophone players and 29 trumpet players. Which band has the higher ratio of trumpet to saxophone players?"""
    junior_sax = 10
    junior_trumpet = 20
    senior_sax = 18
    senior_trumpet = 29
    junior_ratio = junior_trumpet / junior_sax
    senior_ratio = senior_trumpet / senior_sax
    if junior_ratio > senior_ratio:
        result = "junior"
    elif senior_ratio > junior_ratio:
        result = "senior"
    else:
        result = "equal"
    return result

> Finished chain.


'junior'

## Some failures

In [ ]:
# Time - fails
question = "If you wake up at 7:00 a.m. and it takes you 1 hour and 30 minutes to get ready and walk to school, at what time will you get to school?"


In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """If you wake up at 7:00 a.m. and it takes you 1 hour and 30 minutes to get ready and walk to school, at what time will you get to school?"""
    wake_up_time = 7
    minutes_to_get_ready = 90
    minutes_in_an_hour = 60
    hours_to_get_ready = minutes_to_get_ready // minutes_in_an_hour
    total_hours = wake_up_time + hours_to_get_ready
    result = total_hours
    return result

> Finished chain.


'8'

In [ ]:
#ratios - fail
question = "The ratio of Jenny’s trophies to Meredith’s trophies is 7:4.\
 The difference between the numbers is 12. What are the numbers?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
def solution():
    """The ratio of Jenny’s trophies to Meredith’s trophies is 7:4. The difference between the numbers is 12. What are the numbers?"""
    ratio = 7/4
    difference = 12
    jenny_trophies = difference * ratio
    meredith_trophies = difference * (1 - ratio)
    result = (jenny_trophies, meredith_trophies)
    return result

> Finished chain.


'(21.0, -9.0)'

## Colored Objects


In [ ]:
pal_chain = PALChain.from_colored_object_prompt(llm, verbose=True)

In [ ]:
question = "On the desk, you see two blue booklets, two purple booklets, 1 purple hat \
and two yellow pairs of sunglasses.\
 If I remove all the pairs of sunglasses from the desk, how many purple items remain on it?"

In [ ]:
pal_chain.run(question)



> Entering new PALChain chain...
# Put objects into a list to record ordering
objects = []
objects += [('booklet', 'blue')] * 2
objects += [('booklet', 'purple')] * 2
objects += [('hat', 'purple')] * 1
objects += [('sunglasses', 'yellow')] * 2

# Remove all pairs of sunglasses
objects = [object for object in objects if object[0] != 'sunglasses']

# Count number of purple items
num_purple = 0
for object in objects:
    if object[1] == 'purple':
        num_purple += 1

answer = num_purple

> Finished chain.


'3'

## Intermediate steps

In [ ]:
pal_chain = PALChain.from_colored_object_prompt(llm,
                                                verbose=True,
                                                return_intermediate_steps=True)

In [ ]:
question = "On the desk, you see two blue booklets, two purple booklets,\
1 purple hat and two yellow pairs of sunglasses. \
If I remove all the pairs of sunglasses from the desk, how many purple items remain on it?"

In [ ]:
result = pal_chain({"question": question})



> Entering new PALChain chain...
# Put objects into a list to record ordering
objects = []
objects += [('booklet', 'blue')] * 2
objects += [('booklet', 'purple')] * 2
objects += [('hat', 'purple')] * 1
objects += [('sunglasses', 'yellow')] * 2

# Remove all pairs of sunglasses
objects = [object for object in objects if object[0] != 'sunglasses']

# Count number of purple items
num_purple = len([object for object in objects if object[1] == 'purple'])
answer = num_purple

> Finished chain.


In [ ]:
result['intermediate_steps']

"# Put objects into a list to record ordering\nobjects = []\nobjects += [('booklet', 'blue')] * 2\nobjects += [('booklet', 'purple')] * 2\nobjects += [('hat', 'purple')] * 1\nobjects += [('sunglasses', 'yellow')] * 2\n\n# Remove all pairs of sunglasses\nobjects = [object for object in objects if object[0] != 'sunglasses']\n\n# Count number of purple items\nnum_purple = len([object for object in objects if object[1] == 'purple'])\nanswer = num_purple"

# Put objects into a list to record ordering
objects = []
objects += [('booklet', 'blue')] * 2
objects += [('booklet', 'purple')] * 2
objects += [('hat', 'purple')] * 1
objects += [('sunglasses', 'yellow')] * 2

# Remove all pairs of sunglasses
objects = [object for object in objects if object[0] != 'sunglasses']

# Count number of purple items
num_purple = len([object for object in objects if object[1] == 'purple'])
answer = num_purple